In [ ]:
from os import listdir

import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
!aws s3 cp --recursive s3://pert-spectra/figures/ . 

download: s3://pert-spectra/figures/pert_embedding_cluster_gprofiler/pertspectra_inhouse_pert_emb_gprofiler.pickle to pert_embedding_cluster_gprofiler/pertspectra_inhouse_pert_emb_gprofiler.pickle
download: s3://pert-spectra/figures/pert_embedding_cluster_gprofiler/pertspectra_norman_pert_emb_gprofiler.pickle to pert_embedding_cluster_gprofiler/pertspectra_norman_pert_emb_gprofiler.pickle
download: s3://pert-spectra/figures/factor_enrichments/GSFA_norman_factor_enrichment.pickle to factor_enrichments/GSFA_norman_factor_enrichment.pickle
download: s3://pert-spectra/figures/pert_embedding_cluster_gprofiler/GSFA_norman_pert_emb_gprofiler.pickle to pert_embedding_cluster_gprofiler/GSFA_norman_pert_emb_gprofiler.pickle
download: s3://pert-spectra/figures/factor_enrichments/GSFA_inhouse_factor_enrichment.pickle to factor_enrichments/GSFA_inhouse_factor_enrichment.pickle
download: s3://pert-spectra/figures/pert_embedding_cluster_gprofiler/GSFA_inhouse_pert_emb_gprofiler.pickle to pert_embeddi

In [12]:
reconstruction_spearman_files = listdir("reconstruction_spearmans/")
reconstruction_spearman_dict = {}
for fname in reconstruction_spearman_files:
    method = fname.split("_")[0]
    if method not in reconstruction_spearman_dict:
        reconstruction_spearman_dict[method] = {}
    dataset = fname.split("_")[1]
    perf = np.mean(
        [
            float(i)
            for i in open(f"reconstruction_spearmans/{fname}", "r")
            .read()
            .strip()
            .split("\n")
        ]
    )
    reconstruction_spearman_dict[method][dataset] = perf
spearman_df = pd.DataFrame.from_dict(reconstruction_spearman_dict).transpose()
spearman_df.reset_index(inplace=True)
spearman_df.rename(columns={"index": "model"}, inplace=True)
spearman_df = spearman_df.melt("model", var_name="Dataset", value_name="Spearman")
spearman_df.loc[spearman_df["model"] == "pertspectra", "model"] = "PertSpectra"

In [13]:
runtimes = pd.read_csv("runtime/time_complexity_cells.txt", header=0, sep=",")
runtimes = runtimes.loc[runtimes["cells"] == 10000]
runtimes["time"] = -1 * np.log10(runtimes["time"])
runtimes["Dataset"] = "norman"
runtimes2 = runtimes.copy()
runtimes2["Dataset"] = "replogle"
runtimes3 = runtimes.copy()
runtimes3["Dataset"] = "inhouse"
runtimes = pd.concat([runtimes, runtimes2, runtimes3])
runtimes.drop(columns=["cells", "perturbations"], inplace=True)

In [14]:
runtimes

,model,time,Dataset
3,GSFA,-4.137166,norman
7,scETM,-2.995324,norman
11,PertSpectra,-2.718502,norman
3,GSFA,-4.137166,replogle
7,scETM,-2.995324,replogle
11,PertSpectra,-2.718502,replogle
3,GSFA,-4.137166,inhouse
7,scETM,-2.995324,inhouse
11,PertSpectra,-2.718502,inhouse


In [15]:
reconstruction_hypergeo_files = listdir("process_recovery_hypergeo_pvals/")
reconstruction_hypergeo_dict = {}
for fname in reconstruction_hypergeo_files:
    method = fname.split("_")[0]
    if method not in reconstruction_hypergeo_dict:
        reconstruction_hypergeo_dict[method] = {}
    dataset = fname.split("_")[1]
    perf = pd.read_csv(f"process_recovery_hypergeo_pvals/{fname}", header=None, sep=",")
    print(perf.head())
    perf[1] = -1 * np.log10(perf[1])
    perf = np.mean(perf[1])
    reconstruction_hypergeo_dict[method][dataset] = perf
hypergeo_df = pd.DataFrame.from_dict(reconstruction_hypergeo_dict).transpose()
hypergeo_df.reset_index(inplace=True)
hypergeo_df.rename(columns={"index": "model"}, inplace=True)
hypergeo_df = hypergeo_df.melt("model", var_name="Dataset", value_name="hypergeo")
hypergeo_df.loc[hypergeo_df["model"] == "pertspectra", "model"] = "PertSpectra"

         0         1
0  ATP6AP1  1.000000
1    RPL30  1.000000
2    DDX21  0.000379
3      PHB  0.025197
4    DMAP1  0.100606
         0             1
0      AHR  2.638282e-11
1   ARID1A  1.923624e-01
2     BAK1  1.453943e-04
3  BCL2L11  6.373444e-05
4      CBL  2.470848e-04
         0             1
0      AHR  7.461950e-07
1   ARID1A  6.365297e-06
2     BAK1  8.785246e-04
3  BCL2L11  1.197726e-02
4      CBL  2.916654e-04
        0         1
0   RIPK1  0.000001
1  MAP3K7  1.000000
2   IKBKB  0.000002
3    SKP1  1.000000
4   BIRC3  0.014625
        0             1
0    SKP2  5.679505e-05
1  NFKBIA  1.931082e-08
2   TRAF2  1.415568e-15
3   BIRC2  1.293259e-16
4    TAB1  7.861270e-06


In [17]:
reconstruction_f1_files = listdir("pert_embedding_recall_f1")
reconstruction_f1_dict = {}
for fname in reconstruction_f1_files:
    method = fname.split("_")[0]
    if method not in reconstruction_f1_dict:
        reconstruction_f1_dict[method] = {}
    dataset = fname.split("_")[1]
    perf = pd.read_csv(
        f"pert_embedding_recall_f1/{fname}", header=0, index_col=0, sep=","
    )
    perf = perf.loc["StringDB", "F1"]
    print(perf)
    reconstruction_f1_dict[method][dataset] = perf
f1_df = pd.DataFrame.from_dict(reconstruction_f1_dict).transpose()
f1_df.reset_index(inplace=True)
f1_df.rename(columns={"index": "model"}, inplace=True)
f1_df = f1_df.melt("model", var_name="Dataset", value_name="F1")
f1_df.loc[f1_df["model"] == "pertspectra", "model"] = "PertSpectra"

0.2785192646688491
0.8294573643410853
0.8203125
0.6979655712050078
0.2461233916199274
0.8203125
0.6796407185628742
0.6895475819032763


In [18]:
reconstruction_aucpr_files = listdir("pert_embedding_recall_aucpr")
reconstruction_aucpr_dict = {}
for fname in reconstruction_aucpr_files:
    method = fname.split("_")[0]
    if method not in reconstruction_aucpr_dict:
        reconstruction_aucpr_dict[method] = {}
    dataset = fname.split("_")[1]
    perf = pd.read_csv(
        f"pert_embedding_recall_aucpr/{fname}", header=0, index_col=0, sep=","
    )
    perf = perf.loc["StringDB", "AUC"]
    print(perf)
    reconstruction_aucpr_dict[method][dataset] = perf
aucpr_df = pd.DataFrame.from_dict(reconstruction_aucpr_dict).transpose()
aucpr_df.reset_index(inplace=True)
aucpr_df.rename(columns={"index": "model"}, inplace=True)
aucpr_df = aucpr_df.melt("model", var_name="Dataset", value_name="auPRC")
aucpr_df.loc[aucpr_df["model"] == "pertspectra", "model"] = "PertSpectra"

0.7530475544691059
0.685748863225066
0.215161882377014
0.2525738309904994
0.7260584258594132
0.7327761570507714
0.675809046108318
0.5736430637940849


In [35]:
# combine the metrics into 1 df
all_perf = (
    spearman_df.merge(runtimes, on=["model", "Dataset"], how="outer")
    .merge(hypergeo_df, on=["model", "Dataset"], how="outer")
    .merge(f1_df, on=["model", "Dataset"], how="outer")
    .merge(aucpr_df, on=["model", "Dataset"], how="outer")
)
all_perf.rename(
    columns={
        "model": "Model",
        "Spearman": "Spearman R",
        "time": "Runtime -log10(sec)<br>21 perts,10K cells",
        "hypergeo": "Recovery Known Biological Functions<br>Hypergeometric Test -log10(FDR)",
        "F1": "Recall:F1",
        "auPRC": "Recall:auPRC",
    },
    inplace=True,
)
all_perf.loc[all_perf["Dataset"] == "norman", "Dataset"] = "Norman"
all_perf.loc[all_perf["Dataset"] == "inhouse", "Dataset"] = "Inhouse"
all_perf.loc[all_perf["Dataset"] == "replogle", "Dataset"] = "Replogle"

for column in all_perf.columns[2::]:
    all_perf[column] = (all_perf[column] - all_perf[column].min()) / (
        all_perf[column].max() - all_perf[column].min()
    )

In [36]:
all_perf.fillna(0, inplace=True)

In [37]:
all_perf_melted = all_perf.melt(id_vars=["Model", "Dataset"], var_name="Metric")

In [38]:
df = all_perf_melted
fig = px.line_polar(
    df,
    r="value",
    theta="Metric",
    color="Model",
    line_dash="Dataset",
    line_close=True,
    color_discrete_sequence=["#ff0000", "#00ff00", "#0000ff"],
    direction="counterclockwise",
    start_angle=45,
    template="plotly_white",
)
fig.show()

In [39]:
help(px.line_polar)

Help on function line_polar in module plotly.express._chart_types:

line_polar(data_frame=None, r=None, theta=None, color=None, line_dash=None, hover_name=None, hover_data=None, custom_data=None, line_group=None, text=None, symbol=None, animation_frame=None, animation_group=None, category_orders=None, labels=None, color_discrete_sequence=None, color_discrete_map=None, line_dash_sequence=None, line_dash_map=None, symbol_sequence=None, symbol_map=None, markers=False, direction='clockwise', start_angle=90, line_close=False, line_shape=None, render_mode='auto', range_r=None, range_theta=None, log_r=False, title=None, template=None, width=None, height=None) -> plotly.graph_objs._figure.Figure
        In a polar line plot, each row of `data_frame` is represented as vertex
        of a polyline mark in polar coordinates.
        
    Parameters
    ----------
    data_frame: DataFrame or array-like or dict
        This argument needs to be passed for column names (and not keyword
        name